## IDLS Project

# Effects of Regularization Techniques on Image Classification Tasks

Dataset: Cifar10

Model: ResNet50

Batchnorm Layers: 0

References:

https://towardsdatascience.com/understand-and-implement-resnet-50-with-tensorflow-2-0-1190b9b52691

https://github.com/suvoooo/Learn-TensorFlow/blob/master/resnet/Implement_Resnet_TensorFlow.ipynb

In [5]:
import matplotlib.pyplot as plt
from matplotlib.patches import  Rectangle
import tensorflow as tf

import os
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, MaxPool2D,MaxPooling2D, Flatten,BatchNormalization, Dropout,ZeroPadding2D, AveragePooling2D, Add, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras import activations
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

from tensorflow.keras.datasets import cifar10
from keras.utils import to_categorical

import time
import pickle

In [4]:
#Define the Model
#Removing all regularizers

def res_identity(x, filters, num_batchnorm = 0, num_dropout=0, dropout_prob = 0):
  #renet block where dimension doesnot change.
  #The skip connection is just simple identity conncection
  #we will have 3 blocks and then input will be added

    x_skip = x # this will be used for addition with the residual block 
    f1, f2 = filters
    bn = num_batchnorm
    drp = num_dropout

    #first block 
    x = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    if bn>0:
        x = BatchNormalization()(x)
        bn-=1
    x = Activation(activations.relu)(x)
    if drp>0:
        x = Dropout(dropout_prob)(x)
        drp-=1

    #second block # bottleneck (but size kept same with padding)
    x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    if bn>0:
        x = BatchNormalization()(x)
        bn-=1
    x = Activation(activations.relu)(x)
    if drp>0:
        x = Dropout(dropout_prob)(x)
        drp-=1

    # third block activation used after adding the input
    x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    if bn>0:
        x = BatchNormalization()(x)
        bn-=1
    x = Activation(activations.relu)(x)
    if drp>0:
        x = Dropout(dropout_prob)(x)
        drp-=1

    # add the input 
    x = Add()([x, x_skip])
    x = Activation(activations.relu)(x)


    return x

def res_conv(x, s, filters, num_batchnorm = 0, num_dropout=0, dropout_prob = 0):
    x_skip = x
    f1, f2 = filters
    bn = num_batchnorm
    drp = num_dropout

    # first block
    x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid')(x)
    # when s = 2 then it is like downsizing the feature map
    if bn>0:
        x = BatchNormalization()(x)
        bn-=1
    x = Activation(activations.relu)(x)
    if drp>0:
        x = Dropout(dropout_prob)(x)
        drp-=1

    # second block
    x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    if bn>0:
        x = BatchNormalization()(x)
        bn-=1
    x = Activation(activations.relu)(x)
    if drp>0:
        x = Dropout(dropout_prob)(x)
        drp-=1

    #third block
    x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    if bn>0:
        x = BatchNormalization()(x)
        bn-=1

    # shortcut 
    x_skip = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid')(x_skip)
    x_skip = BatchNormalization()(x_skip)

    # add 
    x = Add()([x, x_skip])
    x = Activation(activations.relu)(x)

    return x

def resnet50(num_batchnorm = 0, bn_pooling = False, dropout_prob=0, num_dropout_conv = 0, num_dropout_id = 0, num_dropout=0):

    input_im = Input(shape=(32, 32, 3)) # cifar 10 images size
    x = ZeroPadding2D(padding=(3, 3))(input_im)
    bn = num_batchnorm
    drp = num_dropout

    # 1st stage
    # here we perform maxpooling, see the figure above

    x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(x)
    if bn_pooling:
        x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    if drp>0:
        x = Dropout(dropout_prob)(x)
        drp-=1

    #2nd stage 
    # frm here on only conv block and identity block, no pooling

    x = res_conv(x, s=1, filters=(64, 256),num_batchnorm=bn, num_dropout = num_dropout_conv, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(64, 256),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(64, 256),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)

    # 3rd stage

    x = res_conv(x, s=2, filters=(128, 512),num_batchnorm=bn, num_dropout = num_dropout_conv, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(128, 512),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(128, 512),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(128, 512),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)

    # 4th stage

    x = res_conv(x, s=2, filters=(256, 1024),num_batchnorm=bn, num_dropout = num_dropout_conv, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(256, 1024),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(256, 1024),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(256, 1024),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(256, 1024),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(256, 1024),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)

    # 5th stage

    x = res_conv(x, s=2, filters=(512, 2048),num_batchnorm=bn, num_dropout = num_dropout_conv, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(512, 2048),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)
    x = res_identity(x, filters=(512, 2048),num_batchnorm=bn, num_dropout = num_dropout_id, dropout_prob = dropout_prob)

    # ends with average pooling and dense connection

    x = AveragePooling2D((2, 2), padding='same')(x)

    x = Flatten()(x)
    if drp>0:
        x = Dropout(dropout_prob)(x)
        drp-=1
    x = Dense(10, activation='softmax')(x) #multi-class

    # define the model 

    model = Model(inputs=input_im, outputs=x, name='Resnet50')

    return model

In [6]:
#to measure Time to Accuracy
#https://keras.io/guides/writing_your_own_callbacks/

class timeToAccuracy(keras.callbacks.Callback):
    def __init__(self, startTime, epoch_ckpt, model_name):
        super(timeToAccuracy, self).__init__()
        self.targetAcc = 0.87 #CHANGE TO 0.87 WHEN RUNNING MODEL
        self.foundTarget = False
        self.startTime = startTime
        self.epoch = 0
        self.epoch_ckpt = epoch_ckpt
        self.model_name = model_name
    def on_epoch_end(self, epoch, logs=None):
        if self.epoch % self.epoch_ckpt == 0:
            name = self.model_name + '.h5'
            self.model.save_weights(name)
        self.epoch += 1
        
        if not self.foundTarget:
            if logs['accuracy'] >= self.targetAcc:
                current = time.time()
                print("Time to reach {} accuracy: {} s".format(self.targetAcc, current-self.startTime))
                with open('{}_tta.pkl'.format(model_name), 'wb') as file:
                    pickle.dump(current-self.startTime, file)
                self.foundTarget = True

In [7]:
#Fit, evaluate and checkpoint
def fit_resnet(model, xtrain, ytrain, xtest, ytest, model_name):
  
    EPOCHS = 200      #Change to 200 when running the model
    BATCH_SIZE= 64
    VERBOSITY = 0   #Change to 0 when actually running model
    EPOCH_CKPT = 10 # save model every N epochs

    opt = keras.optimizers.Adam(learning_rate=1e-3)

    model.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=['accuracy'])
    #fit the model
    start = time.time()
    model.fit(
        xtrain,
        ytrain,
        batch_size=BATCH_SIZE,
        verbose=VERBOSITY,
        epochs = EPOCHS,
        callbacks=[timeToAccuracy(startTime=start, epoch_ckpt=EPOCH_CKPT, model_name=model_name)]
    )
    train_time = time.time() - start

    #evaluate
    score = model.evaluate(xtest,ytest)
    loss = score[0]
    acc = score[1]

    return train_time,  acc

In [8]:
#prepare data

(x_train, y_train), (x_test, y_test) = cifar10.load_data() 

# Data normalization
x_train = x_train/255
x_test = x_test/255
y_train = to_categorical(y_train) 
y_test = to_categorical(y_test)

170500096/170498071 [==============================] - 6s 0us/step


In [9]:
dropout_probabilities = [0.2,0.5,0.8]
num_dropout = [0,1,2,3]
num_batchnorm = [0,1,2,3]

In [ ]:
#BN=0
#loop for all combinations
for dp_conv in num_dropout:
    for dp_id in num_dropout:
        if dp_conv or dp_id:
            for prob in dropout_probabilities:
                print('Training BN0, DPID{}, DPCONV{}, DPPROB{}'.format(dp_id, dp_conv, prob))
                model_name = 'model_bn_0_dpid_{}_dpconv_{}_dpprob_{}.pkl'.format(dp_id, dp_conv, prob)
                resnet_model = resnet50(num_batchnorm=0, bn_pooling=True, num_dropout_conv=dp_conv, num_dropout_id=dp_id, dropout_prob=prob)
                tt, acc = fit_resnet(resnet_model,x_train, y_train, x_test, y_test, model_name)
            
                print('Final accuracy {} reached in {}'.format(acc, tt))
        else:
            prob = 0
            print('Training BN0, DPID{}, DPCONV{}, DPPROB{}'.format(dp_id, dp_conv, prob))
            model_name = 'model_bn_0_dpid_{}_dpconv_{}_dpprob_{}.pkl'.format(dp_id, dp_conv, prob)
            resnet_model = resnet50(num_batchnorm=0, bn_pooling=True, num_dropout_conv=dp_conv, num_dropout_id=dp_id, dropout_prob=prob)
            tt, acc = fit_resnet(resnet_model,x_train, y_train, x_test, y_test, model_name)
            
            print('Final accuracy {} reached in {}'.format(acc, tt))

In [18]:
#BN=0
#DropCONV = DropID
num_dropout = [1,2,3]
for dp in num_dropout:
    if dp:
        for prob in dropout_probabilities:
            print('Training BN0, DPID{}, DPCONV{}, DPPROB{}'.format(dp, dp, prob))
            model_name = 'model_bn_0_dpid_{}_dpconv_{}_dpprob_{}.pkl'.format(dp, dp, prob)
            resnet_model = resnet50(num_batchnorm=0, bn_pooling=True, num_dropout_conv=dp, num_dropout_id=dp, dropout_prob=prob)
            tt, acc = fit_resnet(resnet_model,x_train, y_train, x_test, y_test, model_name)
            
            print('Final accuracy {} reached in {}'.format(acc, tt))
    else:
        prob = 0
        print('Training BN0, DPID{}, DPCONV{}, DPPROB{}'.format(dp, dp_conv, prob))
        model_name = 'model_bn_0_dpid_{}_dpconv_{}_dpprob_{}.pkl'.format(dp, dp, prob)
        resnet_model = resnet50(num_batchnorm=0, bn_pooling=True, num_dropout_conv=dp, num_dropout_id=dp, dropout_prob=prob)
        tt, acc = fit_resnet(resnet_model,x_train, y_train, x_test, y_test, model_name)
            
        print('Final accuracy {} reached in {}'.format(acc, tt))

Training BN0, DPID1, DPCONV1, DPPROB0.2
Time to reach 0.87 accuracy: 416.76601004600525 s
313/313 [==============================] - 4s 10ms/step - loss: 1.3709 - accuracy: 0.7403
Final accuracy 0.7402999997138977 reached in 4164.198269844055
Training BN0, DPID1, DPCONV1, DPPROB0.5
Time to reach 0.87 accuracy: 886.6885948181152 s
313/313 [==============================] - 4s 10ms/step - loss: 1.5099 - accuracy: 0.7122
Final accuracy 0.7121999859809875 reached in 4139.772803068161
Training BN0, DPID1, DPCONV1, DPPROB0.8
Time to reach 0.87 accuracy: 1318.508838891983 s
313/313 [==============================] - 4s 10ms/step - loss: 1.9497 - accuracy: 0.6577
Final accuracy 0.6577000021934509 reached in 4186.440722465515
Training BN0, DPID2, DPCONV2, DPPROB0.2
Time to reach 0.87 accuracy: 920.297521352768 s
313/313 [==============================] - 4s 10ms/step - loss: 1.2565 - accuracy: 0.7230
Final accuracy 0.7229999899864197 reached in 4373.524525642395
Training BN0, DPID2, DPCONV2, DP